# Landing Zone para Bronze

## Arquitetura Medallion - Camada Bronze

Este script realiza a ingestão dos dados brutos da Landing Zone (arquivos CSV originais) para a camada Bronze. 

### Operações realizadas:
- Ingestão dos arquivos CSV brutos da Landing Zone
- Validações básicas (schema, tipos de dados)
- Adição de metadados (timestamp de ingestão, fonte)
- Registro de problemas de qualidade sem correção

### Regras da camada Bronze:
- Mantém a granularidade original dos dados
- Não remove dados inválidos, apenas os marca
- Adiciona colunas de controle e auditoria
- Foco na completude, não na qualidade

# Importação de bibliotecas

In [1]:
import os
import glob
import json
import pandas as pd
import sqlite3
from datetime import datetime

# Configuração de Diretórios

In [2]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
BRONZE_DIR = os.path.join(BASE_DIR, 'bronze')

if not os.path.exists(BRONZE_DIR):
    os.makedirs(BRONZE_DIR)

# Carregamento dos dados da camada Bronze

In [3]:
bronze_db_path = os.path.join(BRONZE_DIR, 'bronze_layer.db')
conn = sqlite3.connect(bronze_db_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

print(f"Tabelas disponíveis na camada Bronze: {tables}")

conn.close()

Tabelas disponíveis na camada Bronze: []


# Carregar Tabelas Bronze

In [4]:
def ingest_to_bronze(file_path):
    """Ingestão de dados da landing zone para a camada bronze usando pandas."""
    file_name = os.path.basename(file_path)
    table_name = os.path.splitext(file_name)[0]
    
    print(f"Processando {file_name}...")
    
    try:
        df = pd.read_csv(file_path)
        
        print(f"  - Arquivo carregado: {len(df)} linhas, {len(df.columns)} colunas")
        
        df['_ingestion_timestamp'] = datetime.now().isoformat()
        df['_source_file'] = file_name
        df['_source_system'] = 'olist_kaggle'
        
        null_counts = {col: int(df[col].isnull().sum()) for col in df.columns}
        
        quality_issues = {
            'missing_values': null_counts,
        }
        
        quality_log_path = os.path.join(BRONZE_DIR, f"{table_name}_quality.json")
        with open(quality_log_path, 'w') as f:
            json.dump(quality_issues, f, indent=2)
            
        print(f"  - Metadados adicionados")
        print(f"  - Log de qualidade salvo em {quality_log_path}")
        
        bronze_path = os.path.join(BRONZE_DIR, f"{table_name}.parquet")
        df.to_parquet(bronze_path, index=False)
        print(f"  - Dados salvos na camada bronze: {bronze_path}")
        
        return df
    
    except Exception as e:
        print(f"Erro ao processar {file_name}: {str(e)}")
        return None

# Processamento dos Arquivos

In [5]:
bronze_tables = {}

bronze_db_path = os.path.join(BRONZE_DIR, 'bronze_layer.db')
conn = sqlite3.connect(bronze_db_path)

DATASET_DIR = os.path.join(BASE_DIR, 'dataset')

csv_files = glob.glob(os.path.join(DATASET_DIR, '*.csv'))
print(f"Arquivos CSV encontrados ({len(csv_files)}):")
for csv_file in csv_files:
    print(f"  - {os.path.basename(csv_file)}")

for file_path in csv_files:
    table_name = os.path.splitext(os.path.basename(file_path))[0]
    df = ingest_to_bronze(file_path)
    
    if df is not None:
        bronze_tables[table_name] = df
        
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        
        print(f"  - Dados salvos no SQLite bronze layer: {table_name}")
    
    print("--------------------------------------------------------------------------------")

conn.close()
print("\nProcessamento concluído! {0} tabelas ingeridas para a camada bronze.".format(len(bronze_tables)))

print("\nSalvando metadados...")
bronze_metadata = {
    'tables': list(bronze_tables.keys()),
    'row_counts': {table: len(df) for table, df in bronze_tables.items()},
    'processing_date': datetime.now().isoformat()
}

with open(os.path.join(BRONZE_DIR, 'bronze_metadata.json'), 'w') as f:
    json.dump(bronze_metadata, f, indent=2)

print("Metadata salvo com sucesso!")

Arquivos CSV encontrados (8):
  - olist_customers_dataset.csv
  - olist_orders_dataset.csv
  - olist_order_items_dataset.csv
  - olist_order_payments_dataset.csv
  - olist_order_reviews_dataset.csv
  - olist_products_dataset.csv
  - olist_sellers_dataset.csv
  - product_category_name_translation.csv
Processando olist_customers_dataset.csv...
  - Arquivo carregado: 99441 linhas, 5 colunas
  - Metadados adicionados
  - Log de qualidade salvo em c:\Users\Renan Oliveira\Desktop\data_engineering\bronze\olist_customers_dataset_quality.json
  - Dados salvos na camada bronze: c:\Users\Renan Oliveira\Desktop\data_engineering\bronze\olist_customers_dataset.parquet
  - Dados salvos no SQLite bronze layer: olist_customers_dataset
--------------------------------------------------------------------------------
Processando olist_orders_dataset.csv...
  - Arquivo carregado: 99441 linhas, 8 colunas
  - Metadados adicionados
  - Log de qualidade salvo em c:\Users\Renan Oliveira\Desktop\data_engineering